In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('./data/data.csv')
dup_df = pd.read_csv('./data/updated_ratings.csv')

In [4]:
dup_df

,date,rating,report,problems
0,2019-02-25,high,https://www.sierraavalanchecenter.org/advisory...,"['Wind Slab', 'Storm Slab']"
1,2019-02-25,moderate,https://www.sierraavalanchecenter.org/advisory...,['Wind Slab']
2,2019-02-09,high,https://www.sierraavalanchecenter.org/advisory...,"['Wind Slab', 'Storm Slab']"
3,2019-02-09,considerable,https://www.sierraavalanchecenter.org/advisory...,"['Wind Slab', 'Storm Slab']"
4,2018-03-04,considerable,https://www.sierraavalanchecenter.org/advisory...,"['Deep Slab', 'Wind Slab', 'Loose Wet']"
